## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-22-04-14-00 +0000,wsj,Asian Stocks Mostly Rise as Fears About U.S. T...,https://www.wsj.com/finance/stocks/asian-stock...
1,2026-01-22-04-12-18 +0000,nyt,Live Updates: Trump Touts Greenland Framework ...,https://www.nytimes.com/live/2026/01/21/us/tru...
2,2026-01-22-04-03-58 +0000,nypost,Marine veteran fatally shot at Facebook Market...,https://nypost.com/2026/01/21/us-news/marine-v...
3,2026-01-22-04-00-00 +0000,wsj,"In Ominous Sign for Regime, Iran’s Protests Be...",https://www.wsj.com/world/middle-east/in-omino...
4,2026-01-22-03-57-00 +0000,wsj,The Chinese Spy Machine Infiltrating Taiwan’s ...,https://www.wsj.com/world/asia/taiwan-military...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,98
12,greenland,38
320,davos,32
43,court,19
46,ice,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
316,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,219
161,2026-01-21-20-46-06 +0000,cbc,"Trump reiterates desire to acquire Greenland, ...",https://www.cbc.ca/news/world/trump-davos-spee...,207
9,2026-01-22-03-36-01 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,199
215,2026-01-21-18-53-24 +0000,nypost,"Bessent calls Denmark ‘irrelevant,’ shrugs off...",https://nypost.com/2026/01/21/business/bessent...,189
254,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...,188


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
316,219,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...
94,95,2026-01-21-23-21-00 +0000,wsj,Some Democrats join Republicans on the House O...,https://www.wsj.com/politics/republicans-turn-...
211,79,2026-01-21-19-00-40 +0000,latimes,Supreme Court wary of Trump's bid to fire Fed ...,https://www.latimes.com/politics/story/2026-01...
379,61,2026-01-21-08-38-29 +0000,wsj,"Stock Market News, Jan. 21, 2026: Dow Jumps Af...",https://www.wsj.com/livecoverage/stock-market-...
33,54,2026-01-22-02-17-39 +0000,cbc,Former Uvalde school police officer found not ...,https://www.cbc.ca/news/world/uvalde-police-sc...
254,52,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...
214,49,2026-01-21-18-54-48 +0000,nypost,Barron Trump ‘saved’ friend’s life by calling ...,https://nypost.com/2026/01/21/us-news/barron-t...
342,40,2026-01-21-11-00-00 +0000,latimes,California crypto startup moves to South Dakot...,https://www.latimes.com/business/story/2026-01...
388,39,2026-01-21-05-56-20 +0000,bbc,Photos leaked to BBC show faces of hundreds ki...,https://www.bbc.com/news/articles/c0r4957rq8ro...
239,38,2026-01-21-17-31-24 +0000,nypost,"One dead, 37 injured in Spanish train crash — ...",https://nypost.com/2026/01/21/world-news/anoth...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
